In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import pandas as pd
import csv
import copy
from datetime import datetime, timedelta
from zipfile import ZipFile
from typing import Tuple, Union
import numpy as np
import base64
import itertools
import re

# Load Data

In [ ]:
df_survey = pd.read_csv('census_survey.csv')
df_survey.head()

In [ ]:
# Android UTILS

def standard_format(phone_num):
    '''Converts any phone number format into a standard format. Assumes a valid number is passed (not None, not empty, etc.)'''
    phone_num = re.sub("[^0-9]", "", phone_num) # only keep numeric characters (no plus, space, etc.)
    return phone_num[-9:] # only keep last 9 numbers (no country code or leading zero)

def getBit(num, ix):
    return (num >> ix) & 1


def decode_Base64(binstr):
    '''Takes a binary string and decodes it into bits'''
    for byte in base64.b64decode(binstr):  # (get bytes that each read from lower order bit to higher order bit)
        for i in range(8):
            yield getBit(byte, i)


def getActionType(binary):
    '''loops through the bit generator's next 3 bits to find the action type'''
    if binary.__next__():
        if binary.__next__():
            if binary.__next__():
                return "FILTER"  # 111
            else:
                return "SEARCH"  # 110
        else:
            if binary.__next__():
                return "CONTACT"  # 101
            else:
                return "OPEN BUSINESS SCREEN"  # 100
    else:
        if binary.__next__():
            if binary.__next__():
                return "CALL"  # 011
            else:
                return "UNFAVORITE"  # 010
        else:
            if binary.__next__():  # 001
                return "FAVORITE"
            else:
                return "UNRECOGNIZED ACTION"  # 000


def getDate(binary):
    daycount = 0
    for i in range(12):
        if binary.__next__():
            daycount += pow(2, i)
    reference = datetime.strptime("2022/1/1", "%Y/%m/%d")
    return reference + timedelta(days=daycount)


def getPK(binary):
    pk = 0
    for i in range(16):
        if binary.__next__():
            pk += pow(2, i)
    return pk

def decode_Base64_actions(binstr):
    '''Takes a binary string (b'text') and decodes it into humanly readable action strings'''
    return decode_binary_actions(decode_Base64(binstr))

def decode_binary_actions(binary):
    '''Takes a binary (boolean) generator and decodes it into humanly readable action strings'''
    currentActionType = False
    actionStrings = []
    while binary:
        try:
            if not currentActionType:  # we are starting a new action
                currentActionType = getActionType(binary)
                actionStrings += [currentActionType + " | "]
                if currentActionType == 'UNRECOGNIZED ACTION':
                    actionStrings[-1] += "[000]"
            if currentActionType in ["FAVORITE", "UNFAVORITE", "CALL", "OPEN BUSINESS SCREEN", "CONTACT"]:
                date = getDate(binary)
                actionStrings[-1] += date.strftime("%Y/%m/%d") + " | "
                actionStrings[-1] += "Business pk: " + str(getPK(binary))
                for _ in range(9):
                    binary.__next__()
                currentActionType = False
            elif currentActionType in ["SEARCH", "FILTER"]:
                date = getDate(binary)
                actionStrings[-1] += date.strftime("%Y/%m/%d") + " | "
                actionStrings[-1] += "SUCCESSFUL | " if binary.__next__() else "UNSUCCESSFUL | "
                actionStrings[-1] += "TRUNCATED | " if binary.__next__() else "UNTRUNCATED | "
                actionStrings[-1] += "CLEANED | " if binary.__next__() else "RAW | "
                searchstr = ""
                isOffByThree = False
                while True:
                    value = -1
                    for i in range(5):
                        value += pow(2, i) * binary.__next__()
                    if value == -1:
                        bit1 = binary.__next__()
                        bit2 = binary.__next__()
                        bit3 = binary.__next__()
                        if not bit1 and not bit2 and not bit3 and not (currentActionType == "FILTER" and searchstr in ['mafuta ya', 'kuosha']):
                            binary = itertools.chain([0, 0, 0], binary)
                            break
                        elif len(searchstr) >= 21:
                            isOffByThree = True
                            binary = itertools.chain([bit1, bit2, bit3], binary)
                            break
                        else: # catch misencoded strings that contain special characters
                            binary = itertools.chain([bit1, bit2, bit3], binary)
                            searchstr += " "
                    else:
                        searchstr += chr(value + 97)
                actionStrings[-1] += searchstr
                bits = 3 + len(searchstr) * 5 + 8 + 3 + 12
                for _ in range(3):
                    binary.__next__()
                if bits % 8 != 0:
                    for _ in range(8 - (bits % 8)):
                        binary.__next__()
                currentActionType = False
                if isOffByThree: # recovery mode from off by 3 error
                    currentActionType = "UNRECOGNIZED ACTION"
                    actionStrings += [currentActionType + " | ["]
                    for _ in range(3):
                        actionStrings[-1] += str(binary.__next__())
                    actionStrings[-1] += "]"
            else:
                for _ in range(5):
                    actionStrings[-1] += str(binary.__next__())
                while not actionStrings[-1].endswith("0000000000"):
                    for _ in range(8):
                        actionStrings[-1] += str(binary.__next__())
                currentActionType = False
        except StopIteration:
            break
    return actionStrings

In [ ]:
def decode_binary_file(bytes):
    for byte in bytes: 
        for i in range(8):
            yield getBit(byte, i)

def createRow(number, binaryStr):
    actionStrings = decode_binary_actions(decode_binary_file(binaryStr))
    row = {"phone_number": number, "num_actions": len(actionStrings), 
            "FILTER": 0, "SEARCH": 0, "CONTACT": 0, "OPEN BUSINESS SCREEN": 0, 
            "CALL": 0, "UNFAVORITE": 0, "FAVORITE": 0, "UNRECOGNIZED ACTION": 0, "dates": [], "filter_words": [], "search_words": []}
    for action in actionStrings:
        parts = action.split(' | ')
        row[parts[0]] += 1
        if parts[0] != "UNRECOGNIZED ACTION":
            row['dates'] += [parts[1]]
        if parts[0] == "FILTER":
            row['filter_words'] += [parts[-1]]
        elif parts[0] == "SEARCH":
            row['search_words'] += [parts[-1]]
    return row

In [ ]:
df_android = pd.read_csv('android.csv')
# df_android = pd.DataFrame()
# with ZipFile('logs/android_logs_6_8.zip', 'r') as archive:
#     for name in archive.namelist():
#         if '_decoded' in name or 'Android' not in name:
#             continue
#         number = standard_format(name.split('.')[0].split('_')[1])
#         with archive.open(name) as file:
#             row = createRow(number, file.read())
#             df_android = pd.concat([df_android, pd.DataFrame(row.values(), index=row.keys()).T], ignore_index=True)
df_android.head()

In [ ]:
df_android.to_csv('android.csv', index=False)

In [ ]:
# from ekichabi.whitelist.GetWhiteList import populate_whitelist_for_admins
def populate_whitelist_for_admins(set):
    # demo number for internal testing
    set.add(standard_format('255000000000'))

In [ ]:
# load whitelist
whitelist = {}
with open('../ekichabi-server/data/whitelist_pretty.csv', 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # ignore first row - which contains field names
    next(csvreader)

    # extracting phone number(s) out of each row
    for row in csvreader:
        date = datetime.strptime(row[-1], '%Y-%d-%b')
        if date not in whitelist:
            whitelist[date] = set()
        if str(row[1]):
            whitelist[date].add(standard_format(str(row[1])))  # number1

print(whitelist.keys())
totalwhitelist = set.union(*whitelist.values())
print(len(totalwhitelist))

In [ ]:
# load enumerators
enumerators = set()
with open('../ekichabi-server/data/Enumerator.csv', 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # ignore header
    next(csvreader)

    # extracting phone number out of each row
    for row in csvreader:
        enumerators.add(standard_format(str(row[2])))  # number1
print(enumerators)

In [ ]:
# load testers and admins
testers = set()
populate_whitelist_for_admins(testers)
print(testers)

In [ ]:
# load wakala intervention group
wakalagroup = set()
with open('../ekichabi-server/data/Wakala.csv', 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # extracting phone number out of each row
    for row in csvreader:
        wakalagroup.add(standard_format(str(row[1])))  # number1
print(wakalagroup)

In [ ]:
# load business group
cutoffdate = datetime.strptime('2022-7-Dec', '%Y-%d-%b')
businessgroup = set()
with open('../ekichabi-server/data/census_data_trimmed.csv', 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # ignore first row - which contains field names
    next(csvreader)

    # extracting phone number out of each row
    for row in csvreader:
        businessgroup.add(standard_format(str(row[17])))  # number1
        businessgroup.add(standard_format(str(row[18])))  # number2
businessgroup.remove("")
len(businessgroup)

In [ ]:
def wasWhitelisted(number: str, date: datetime) -> bool:
    number = standard_format(number)
    if number in testers or number in enumerators:
        return False
    if date <= cutoffdate and number in businessgroup:
        return True
    for d, l in whitelist.items():
        if d > date:
            return False
        if number in l:
            return True
    return False

In [ ]:
def categorizeGroup(number, time):
    return 'non-whitelisted' if (not wasWhitelisted(number, time)) else 'wakalagroup' if number in wakalagroup else 'business' if number in businessgroup else 'household'

In [ ]:
sum([1 if categorizeGroup(standard_format(str(n)), cutoffdate) == 'household' else 0 for n in totalwhitelist])

In [ ]:
# load log data
def parseLine(line: bytes) -> Tuple[datetime, str, Union[str, bool]]:
    line = line.decode().strip()
    columns = line.split('\t')
    t = datetime.strptime(columns[0], '[%H:%M:%S.%f]')
    time = startTime + timedelta(hours=t.hour, minutes=t.minute, seconds=t.second, microseconds=t.microsecond)
    action = columns[1]
    details = columns[3] if len(columns) > 3 else False
    return time, action, details

def initRow(number, startTime):
    return {'phone_number': number, 'session_start': startTime, 'session_end': pd.NaT, 'duration': pd.NaT,'inputs': "", 
        'group': categorizeGroup(number, startTime), 
        'businesses_visited': 0, 'businesses': [], 'searches': 0, 'filter_location': 0, 'helper_instruction': 0, 'category_browse': 0, 'location_browse': 0, 'exit': 0, 'home': 0, 'back': 0}

df = pd.read_csv('logs.csv')

# df = pd.DataFrame(columns=initRow('000000000', 0).keys())
# with ZipFile('logs/all_ussd.zip', 'r') as archive:
#     for name in archive.namelist(): # name = [phone_number]_[Year]-[month]-[day]-[Hour].[Minute].[Second]
#         parts = name.split('_')
#         number = standard_format(parts[0])
#         print(name)
#         print(parts[1].replace('.log', ''))
#         startTime = datetime.strptime(parts[1].replace('.log', ''), '%Y-%m-%d-%H.%M.%S')
#         # if not wasWhitelisted(number, startTime):
#         #     continue
#         if number in testers or number in enumerators:
#             continue
#         row = initRow(number, startTime)
#         isHome = True
#         path = []
#         with archive.open(name) as file:
#             file.readline() # ignore session info
#             line = file.readline() # ignore column headers
#             # line = file.readline() # ignore first home screen

#             def handleReadLine():
#                 global line
#                 global row
#                 global isHome
#                 global path
#                 line = file.readline()
#                 if not line:
#                     return "", "", ""
#                 time, action, details = parseLine(line)
#                 row['session_end'] = time
#                 row['duration'] = time - startTime
#                 if action == 'INPUT RECEIVED':
#                     row['inputs'] += ("," if len(row['inputs']) != 0 else "") + details.strip("'")
#                 elif action == 'RENDERED SCREEN' and details == 'SearchSelectorScreen':
#                     row['searches'] += 1
#                 elif action == 'RENDERED SCREEN' and details == 'IfFilterBYLocationScreen':
#                     row['filter_location'] += 1
#                 elif action == 'RENDERED SCREEN' and details == 'HelpScreen':
#                     row['helper_instruction'] += 1
#                 elif isHome and action == 'RENDERED SCREEN' and details == 'MenuHierarchyScreen - selecting a category':
#                     row['category_browse'] += 1
#                 elif isHome and action == 'RENDERED SCREEN' and details == 'MenuHierarchyScreen - selecting a district':
#                     row['category_browse'] += 1
#                 elif action == 'RENDERED SCREEN' and details and details.startswith('BusinessDetailsScreen'):
#                     row['businesses_visited'] += 1
#                     duration = pd.NaT
#                     next_line = file.peek()
#                     if next_line:
#                         next_time, _, _ = parseLine(next_line)
#                         duration = next_time - time
#                     business = {'id': details.split(' - for ')[1], 'duration': duration,'path': copy.copy(path)}
#                     row['businesses'].append(business)
#                 if action == 'RENDERED SCREEN':
#                     path.append(details)
#                     isHome = False
#                 return time, action, details

#             while line:
#                 time, action, details = handleReadLine()
#                 if not line: break
#                 if action == 'BACK PRESSED': # BACK PRESSED + RENDERED SCREEN (back button -- 99)
#                     row['back'] += 1
#                     _, _, details = handleReadLine()
#                     isHome = details == 'HomeScreen'
#                     if isHome:
#                         path = []
#                 elif action == 'HOME PRESSED': # HOME PRESSED + RENDERED SCREEN (home button -- 100)
#                     row['home'] += 1
#                     handleReadLine()
#                     isHome = True
#                     path = []
#                 elif action == 'EXIT PRESSED': # EXIT PRESSED + RENDERED SCREEN (session end button -- 109)
#                     row['exit'] += 1
#                     handleReadLine()
#         df = pd.concat([df, pd.DataFrame(row.values(), index=row.keys()).T], ignore_index=True)
#         # break

#     print(len(archive.namelist()))
pd.set_option('display.max_colwidth', None)
df.head()

In [ ]:
df.to_csv('logs.csv')

In [ ]:
df.info()

In [ ]:
df_whitelisted = df[df['group'] != 'non-whitelisted']

# Data Viz Utils

In [ ]:
def hist(df, cat = 'wakala_frequency', labels = {1: '1 time or less', 2: '2-5 times', 3:'5+ times'}):
    val = 'percentage'
    temp = pd.DataFrame({cat:[labels[v] for v in df[cat].value_counts().keys()], val:df[cat].value_counts()})
    temp[val] = temp[val].apply(lambda x: (x/temp[val].sum())*100)
    labels = temp[val].round(1).astype('str') + '%'
    ax = temp.plot.bar(x=cat, y=val, rot=30)
    for container in ax.containers:
        ax.bar_label(container, labels=labels)
        ax.yaxis.set_major_formatter(mtick.PercentFormatter())

In [ ]:
def categorize1vs1234(number):
    try:
        number = standard_format(str(int(number)))
        labels = {
            'Very Comfortable--I can use USSD without help from anyone': 1, 
            'Somewhat comfortable--I may ask for help sometimes': 2, 
            'Not very comfortable--I need help over 50% of the time': 3, 
            'Uncomfortable--I always ask someone else to navigate the menus on my behalf': 4
        }
        match = df_survey[df_survey['phone_number'].apply(lambda n: str(standard_format(str(n)))) == number]['comfort_with_ussd']
        return labels[list(match)[0] if isinstance(match, pd.Series) else match] 
    except:
        return -1

In [ ]:
def stackedBarChart(df, horizCat, stackedCat):
    df2 = df.groupby([horizCat, stackedCat])[horizCat].count().unstack(stackedCat).fillna(0)
    df2[df[stackedCat].unique()].plot(kind='bar', stacked=True)

# Analysis

In [ ]:
df_android.info()

In [ ]:
df_android["group"] = df_android["phone_number"].apply(lambda n: categorizeGroup(str(n), cutoffdate))

In [ ]:
df_android.drop_duplicates(subset=['phone_number'], keep='first', inplace=False).value_counts('group')

In [ ]:
sum(df_android['num_actions'])

In [ ]:
874 / 40

In [ ]:
df_whitelisted['phone_number'].nunique()

In [ ]:
sum(df_whitelisted.apply(lambda n: 0 if not isinstance(n['inputs'], str) else n['inputs'].count(','), axis=1))

In [ ]:
24318 / 578

In [ ]:
sum(df_whitelisted['searches'])

In [ ]:
288 / 578

In [ ]:
sum(df_android['SEARCH'])

In [ ]:
85 / 40

In [ ]:
sum(df_android.apply(lambda n: 0 if not isinstance(n['dates'], list) else len(set(n['dates'])), axis=1)) / 40

In [ ]:
sum(df_whitelisted.groupby('phone_number').apply(lambda n: n['session_start'].apply(lambda x: x[:10]).nunique())) / 578

In [ ]:
len(set.union(*[set([el for el in lis]) for lis in df_android['filter_words']]))

In [ ]:
pd.DataFrame([el for lis in df_android['filter_words'] for el in lis], columns=['filter_words']).value_counts()[:10].plot.bar()

In [ ]:
pd.DataFrame([el for lis in df_android['search_words'] for el in lis], columns=['search_words']).value_counts()

In [ ]:
df_android.head(1)

In [ ]:
df_nonanon = pd.read_csv('hh_nonanon.csv').drop('Unnamed: 0', axis=1)
df_nonanon = df_nonanon[df_nonanon["digital"].apply(lambda x: bool(x))]
def classify_phone(row):
    if row["p_basic"] == 1 and row["p_smart"] == 1:
        return "Both" 
    elif row["p_basic"] == 1:
        return  "Feature"
    elif row["p_smart"] == 1:
        return "Smart"
    else:
        return "None"
df_nonanon["phone_type"] = df_nonanon.apply(classify_phone, axis=1)
df_nonanon['age_group'] = pd.cut(df_nonanon['age'], bins=[0, 18, 25, 35, 45, 55, 65, 100])
df_nonanon['f_104_ussd'] = df_nonanon['f_104_ussd'].replace({
    "Very Comfortable – I can use USSD without help from anyone": "1. Very Comfortable",
    "Somewhat comfortable – I may ask for help sometimes": "2. Somewhat Comfortable",
    "Not very comfortable – I need help over 50% of the time": "3. Not Comfortable",
    "Uncomfortable – I always ask someone else to navigate the menus on my behalf": "4. Very Uncomfortable"
})
df_nonanon.loc[df_nonanon['f_103_ussd'] == 'No', 'f_104_ussd'] = '0. Doesn\'t Use USSD'
df_nonanon.head()

In [ ]:
set(df_nonanon["phone_number"].fillna(0).astype("int").astype("str")) & set(df_android["phone_number"])

In [ ]:
df_nonanon[df_nonanon["phone_number"] == 782809975]["f_104_ussd"]

In [ ]:
sum(df_android["FAVORITE"] != 0)

In [ ]:
5/40